<a href="https://colab.research.google.com/github/nadirbelarouci/NN-overfitting-isgood/blob/master/NN_overfitting_is_good.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
INPUT_LAYER_SIZE = 32
LABELS_SIZE = 27

def prepare_data():
  random_text = "BASED ON YOUR INPUT GET A RANDOM ALPHA NUHE RANDOM STRING GENERATOR CREATES A SERIES OF AND LETTERS ALRBRFNKPZNJEBPCZ FHGDQPZLHE HELLO THERE HOW ARE YOU DOING TODAY AND BLA BLA BLA"
  
  length = len(random_text)
  
  x_train = np.zeros((INPUT_LAYER_SIZE,length))
  y_train = np.zeros(length)                   
  
  for i in range(length):
    
    x = [int(x) for x in bin(i)[2:]] 
    x = np.pad(x,(32-len(x),0),'constant')
    
    x_train[:,i] = x
    y_train[i] = LABELS_SIZE-1 if random_text[i]==' ' else ord(random_text[i]) - 65
   
    
  x_train = x_train.T
  y_train = y_train.T 
  
  return x_train, y_train

In [3]:
x_train, y_train = prepare_data()
print(x_train.shape,y_train.shape)
print(np.min(y_train))

(180, 32) (180,)
0.0


In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(LABELS_SIZE, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
# Train neural network
score = 0.0
while score < 1.0:
  history = model.fit(x_train, y_train, epochs=5,verbose=False)
  l = len(history.history['acc'])-1
  score = history.history['acc'][l];

In [0]:
score = model.evaluate(x_train, y_train)
print(score)


In [38]:

    
predictions = np.argmax(model.predict(x_train),axis=1) 

to_char = lambda x: " " if x == LABELS_SIZE-1 else chr(x+65)
text = "".join(list(map(to_char, predictions)))

print (text)

BASED ON YOUR INPUT GET A RANDOM ALPHA NUHE RANDOM STRING GENERATOR CREATES A SERIES OF AND LETTERS ALRBRFNKPZNJEBPCZ FHGDQPZLHE HELLO THERE HOW ARE YOU DOING TODAY AND BLA BLA BLA
